<a href="https://colab.research.google.com/github/Xboster-eliel/bosterpy/blob/main/funciones_folio_personales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import os

import zipfile
import folium
from folium import plugins
from folium.plugins import Draw
from folium.plugins import MeasureControl
from folium.plugins import Fullscreen
from pyproj import Transformer
from pyproj import CRS

## MODIFICANDO LA CLASE DRAW DE *FOLIUM*

In [ ]:
!pip install folium --upgrade  # Asegurar versión reciente

import folium
from branca.element import Template, MacroElement
from folium.elements import JSCSSMixin

In [ ]:
class CustomDraw(JSCSSMixin, MacroElement):
    _template = Template("""
        {% macro html(this, kwargs) %}
            <style>
                #export {
                    position: absolute;
                    top: 90px;
                    right: 10px;
                    z-index: 999;
                    background: white;
                    color: black;
                    padding: 6px;
                    border-radius: 4px;
                    font-family: 'Helvetica Neue';
                    cursor: pointer;
                    font-size: 12px;
                    text-decoration: none;
                }
            </style>
            <a href='#' id='export'>Export</a>
        {% endmacro %}

        {% macro script(this, kwargs) %}
            var options = {
                position: {{ this.position|tojson }},
                draw: {{ this.draw_options|tojson }},
                edit: {{ this.edit_options|tojson }},
            };

            var drawnItems_{{ this.get_name() }} = new L.featureGroup().addTo({{ this._parent.get_name() }});
            options.edit.featureGroup = drawnItems_{{ this.get_name() }};

            var {{ this.get_name() }} = new L.Control.Draw(options).addTo({{ this._parent.get_name() }});

            {{ this._parent.get_name() }}.on(L.Draw.Event.CREATED, function(e) {
                var layer = e.layer;
                var type = e.layerType;

                // Aplicar estilos personalizados
                if (type === 'polyline') {
                    layer.setStyle({
                        color: 'blue',
                        weight: 3,
                        opacity: 0.7
                    });
                    var distance = L.GeometryUtil.length(layer.getLatLngs());
                    layer.bindPopup('Distancia: ' + distance.toFixed(2) + ' metros');
                }

                if (type === 'marker') {
                    layer.setIcon(L.icon({
                        iconUrl: 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-2x-blue.png',
                        iconSize: [25, 41],
                        iconAnchor: [12, 41],
                        popupAnchor: [1, -34]
                    }));
                    var coords = layer.getLatLng();
                    layer.bindPopup('Coordenadas: ' + coords.lat.toFixed(4) + ', ' + coords.lng.toFixed(4));
                }

                if (type === 'rectangle') {
                    layer.setStyle({
                        color: 'green',
                        fillColor: 'yellow',
                        fillOpacity: 0.3,
                        weight: 2
                    });
                    var area = L.GeometryUtil.geodesicArea(layer.getLatLngs()[0]);
                    layer.bindPopup('Área: ' + (area/10000).toFixed(2) + ' hectáreas');
                }

                if (type === 'polygon') {
                    layer.setStyle({
                        color: 'red',
                        fillColor: 'yellow',
                        fillOpacity: 0.3,
                        weight: 2
                    });
                    var area = L.GeometryUtil.geodesicArea(layer.getLatLngs()[0]);
                    layer.bindPopup('Área: ' + (area/10000).toFixed(2) + ' hectáreas');
                }

                if (type === 'circle') {
                    layer.setStyle({
                        color: 'red',
                        fillColor: 'green',
                        fillOpacity: 0.3,
                        weight: 2
                    });
                    layer.bindPopup('Radio: ' + layer.getRadius().toFixed(2) + ' metros');
                }

                if (type === 'circlemarker') {
                    layer.setStyle({
                        color: 'blue',
                        fillColor: 'yellow',
                        fillOpacity: 0.6,
                        radius: 10
                    });
                    var coords = layer.getLatLng();
                    layer.bindPopup('Coordenadas: ' + coords.lat.toFixed(4) + ', ' + coords.lng.toFixed(4));
                }

                drawnItems_{{ this.get_name() }}.addLayer(layer);
            });

            document.getElementById('export').onclick = function() {
                var data = drawnItems_{{ this.get_name() }}.toGeoJSON();
                var blob = new Blob([JSON.stringify(data)], {type: 'application/json'});
                var url = window.URL.createObjectURL(blob);
                var a = document.createElement('a');
                a.href = url;
                a.download = {{ this.filename|tojson }};
                document.body.appendChild(a);
                a.click();
                document.body.removeChild(a);
            };
        {% endmacro %}
    """)

    default_js = [
        ('leaflet_draw_js', 'https://cdnjs.cloudflare.com/ajax/libs/leaflet.draw/1.0.4/leaflet.draw.js'),
        ('leaflet_geometry_util', 'https://cdn.jsdelivr.net/npm/leaflet-geometryutil@0.9.3/src/leaflet.geometryutil.min.js')
    ]

    default_css = [
        ('leaflet_draw_css', 'https://cdnjs.cloudflare.com/ajax/libs/leaflet.draw/1.0.4/leaflet.draw.css')
    ]

    def __init__(self, export=True, filename='datos.geojson', position='topleft',
                 draw_options=None, edit_options=None):
        super().__init__()
        self._name = 'CustomDraw'
        self.export = export
        self.filename = filename
        self.position = position
        self.draw_options = draw_options or {
            'circle': {'showRadius': True, 'metric': True},
            'circlemarker': {}
        }
        self.edit_options = edit_options or {'edit': True, 'remove': True}

In [ ]:
import folium

def crear_mapa(location=[19.4326, -99.1332], zoom_start=12, exportar_datos=False):
    """
    Crea un mapa interactivo con control de dibujo personalizado usando CustomDraw.

    Parámetros:
    location (list): Coordenadas [lat, lon] para centrar el mapa (por defecto: CDMX)
    zoom_start (int): Nivel de zoom inicial (por defecto: 12)
    exportar_datos (bool): Si True, devuelve función para obtener datos dibujados

    Retorna:
    folium.Map: Mapa interactivo
    function (opcional): Función para obtener datos si exportar_datos=True
    """

    # Crear mapa base
    m = folium.Map(location=location, zoom_start=zoom_start)

    # Crear FeatureGroup para almacenar los dibujos
    feature_group = folium.FeatureGroup(name="Elementos Dibujados").add_to(m)

    # Instanciar control de dibujo personalizado
    draw_control = CustomDraw(
        export=True,
        filename="mis_dibujos.geojson",
        position="topleft",
        draw_options={
            'circle': {'shapeOptions': {'color': '#ff0000'}},
            'circlemarker': {'shapeOptions': {'radius': 15}},
            'polygon': {'allowIntersection': False},
            'rectangle': {'showArea': True}
        },
        edit_options={'edit': True, 'remove': True}
    )
    draw_control.add_to(m)

    # Función para exportar datos (versión simplificada: solo operativa si se usan capas de Python)
    def obtener_datos():
        datos = {
            'circulos': [],
            'circlemarkers': [],
            'poligonos': [],
            'rectangulos': []
        }

        for capa in feature_group._children.values():
            if isinstance(capa, folium.Circle):
                datos['circulos'].append({
                    'centro': capa.location,
                    'radio': capa.radius,
                    'color': capa.options.get('color', '#ff0000')
                })
            elif isinstance(capa, folium.CircleMarker):
                datos['circlemarkers'].append({
                    'centro': capa.location,
                    'radio': capa.options.get('radius', 15),
                    'color': capa.options.get('color', '#3388ff')
                })
            elif isinstance(capa, folium.Polygon):
                datos['poligonos'].append({
                    'coordenadas': capa.locations,
                    'color': capa.options.get('color', '#3388ff')
                })
            elif isinstance(capa, folium.Rectangle):
                datos['rectangulos'].append({
                    'limites': capa.bounds,
                    'color': capa.options.get('color', '#3388ff')
                })

        return datos

    return (m, obtener_datos) if exportar_datos else m

## FUNCION PARA LA CARGA DE GEOPAKETES PROVENIENTES DE AVENZA MAP

In [ ]:
import folium
import geopandas as gpd
from html import escape
from base64 import b64encode

def agregar_fotos_mapa(mapa_existente, geopackage_path):
    """
    Agrega fotos desde un GeoPackage a un mapa Folium existente.

    Parámetros:
    mapa_existente (folium.Map): Mapa al que se agregarán las fotos
    geopackage_path (str): Ruta al archivo GeoPackage

    Retorna:
    folium.Map: Mapa actualizado con las fotos
    """

    # Preguntar por la capa a usar
    layer_input = input("¿La capa sigue siendo 'Photos'? (Enter para sí o ingrese nuevo nombre): ")
    layer = layer_input if layer_input.strip() else 'Photos'

    try:
        # Cargar el GeoDataFrame
        gdf = gpd.read_file(geopackage_path, layer=layer)

        # Verificar existencia de columna 'photo'
        if 'photo' not in gdf.columns:
            raise ValueError(f"La capa '{layer}' no contiene la columna 'photo'")

        # Función interna para crear popups seguros
        def crear_popup(fila):
            try:
                # Manejar diferentes tipos de datos en 'photo'
                if isinstance(fila['photo'], bytes):
                    img_b64 = b64encode(fila['photo']).decode('utf-8')
                else:
                    ruta = fila['photo'].replace("\\", "/")
                    with open(ruta, "rb") as f:
                        img_b64 = b64encode(f.read()).decode('utf-8')

                img_html = f'<img src="data:image/jpeg;base64,{img_b64}" style="max-width:250px;">'

            except Exception as e:
                img_html = f'<p>Error cargando imagen: {escape(str(e))}</p>'

            return f"""
            <div style="width:300px;">
                {img_html}
                <p><b>Observaciones:</b> {escape(str(fila['remarks']))}</p>
                <p><b>Fecha:</b> {escape(str(fila['time']))}</p>
                <p><b>Coordenadas:</b> {fila.geometry.y:.4f}, {fila.geometry.x:.4f}</p>
            </div>
            """

        # Añadir marcadores al mapa
        for _, fila in gdf.iterrows():
            folium.Marker(
                location=[fila.geometry.y, fila.geometry.x],
                popup=folium.Popup(crear_popup(fila), max_width=350),
                icon=folium.Icon(icon='camera', color='blue', prefix='fa')
            ).add_to(mapa_existente)

        print(f"Se agregaron {len(gdf)} fotos al mapa")
        return mapa_existente

    except Exception as e:
        print(f"Error: {str(e)}")
        return mapa_existente

## FUNCION PARA CARGAR | .shp | .zip | .xlxs | , GUARDAR EN UNA RUTA Y RECORDAR LOS ARCHIVOS ALMACENADOS

In [ ]:
def añadir_shape(mapa, ruta_almacenamiento='capas_guardadas'):
    """
    Añade shapefiles/ZIPs/Excel a un mapa Folium con capacidad de almacenamiento y reutilización de capas

    Parámetros:
    mapa (folium.Map): Mapa objetivo donde se añadirán las capas
    ruta_almacenamiento (str): Ruta para guardar las capas cargadas
    """
    import geopandas as gpd
    import zipfile
    import os
    from io import BytesIO
    import requests
    import pandas as pd
    import tempfile
    import folium
    from shapely.geometry import Point

    # Configurar directorio de almacenamiento
    os.makedirs(ruta_almacenamiento, exist_ok=True)

    # Configuración de estilos guardados
    ESTILOS_GUARDADOS = {
        'cuerpo_agua': {'fillColor': '#0000FF', 'color': '#0000FF'},
        'amenaza_alta': {'fillColor': '#ff0000', 'color': '#4a0d0d'},
        'amanaza_baja': {'fillColor': '#00ff00', 'color': '#0d4a0d'},
        'amenaza_media': {'fillColor': '#fbfcfc', 'color': '#000000'}
    }

    def cargar_shapefile(ruta, contenido=None):
        """Carga datos desde múltiples formatos"""
        try:
            # Manejar contenido en memoria primero
            if contenido:
                return cargar_desde_memoria(ruta, contenido)

            # Detectar tipo de archivo por extensión
            if ruta.endswith('.zip'):
                return cargar_shapefile_zip(ruta)
            elif ruta.endswith(('.xls', '.xlsx')):
                return cargar_excel(ruta)
            else:
                return gpd.read_file(ruta)

        except Exception as e:
            print(f"Error cargando {ruta}: {str(e)}")
            return None

    def procesar_zip(zip_ref):
        """Procesa archivos ZIP"""
        temp_dir = tempfile.mkdtemp()
        zip_ref.extractall(temp_dir)

        shp_files = [f for f in zip_ref.namelist() if f.endswith('.shp')]
        if not shp_files:
            raise ValueError("ZIP no contiene .shp")

        gdf = gpd.read_file(os.path.join(temp_dir, shp_files[0]))

        # Buscar metadatos de estilo
        if 'estilos.csv' in zip_ref.namelist():
            estilos_df = pd.read_csv(zip_ref.open('estilos.csv'))
            return gpd.GeoDataFrame(gdf.merge(estilos_df, on='nombre'))

        return gdf

    def cargar_desde_memoria(ruta, contenido):
        """Maneja archivos subidos desde Drive/Colab"""
        with tempfile.TemporaryDirectory() as tmpdir:
            file_path = os.path.join(tmpdir, ruta)

            if ruta.endswith('.zip'):
                with zipfile.ZipFile(BytesIO(contenido)) as zip_ref:
                    return procesar_zip(zip_ref)
            elif ruta.endswith(('.xls', '.xlsx')):
                with open(file_path, 'wb') as f:
                    f.write(contenido)
                return cargar_excel(file_path)
            else:
                with open(file_path, 'wb') as f:
                    f.write(contenido)
                return gpd.read_file(file_path)

    def cargar_shapefile_zip(ruta_zip):
        """Procesa archivos ZIP con validación mejorada"""
        print(f"\nProcesando archivo ZIP: {ruta_zip}")

        with zipfile.ZipFile(ruta_zip) as zip_ref:
            # Listar contenido y capas disponibles
            print("Contenido del ZIP:")
            print(zip_ref.namelist())

            # Extraer a directorio temporal
            temp_dir = tempfile.mkdtemp()
            zip_ref.extractall(temp_dir)

            # Buscar archivos shapefile
            shp_files = [f for f in zip_ref.namelist() if f.endswith('.shp')]
            if not shp_files:
                raise ValueError("ZIP no contiene archivos .shp válidos")

            # Cargar primer shapefile encontrado
            gdf = gpd.read_file(os.path.join(temp_dir, shp_files[0]))

            # Guardar copia local
            nombre_capa = os.path.splitext(os.path.basename(shp_files[0]))[0]
            # guardar_capa(gdf, nombre_capa)

            return gdf

    def cargar_excel(ruta_excel):
        """Carga y convierte archivos Excel a GeoDataFrame"""
        print(f"\nProcesando archivo Excel: {ruta_excel}")

        # Leer archivo Excel
        df = pd.read_excel(ruta_excel)

        # Solicitar columnas de coordenadas
        print("Columnas disponibles:", df.columns.tolist())
        lon_col = input("Columna para longitud: ") or 'LONGITUD'
        lat_col = input("Columna para latitud: ") or 'LATITUD'

        # Crear geometrías
        geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

        # Guardar copia local
        nombre_capa = os.path.splitext(os.path.basename(ruta_excel))[0]
        # guardar_capa(gdf, nombre_capa)

        return gdf

    def guardar_capa(gdf, nombre):
        """Almacena capas para reutilización futura"""
        ruta_guardado = os.path.join(ruta_almacenamiento, f"{nombre}.geojson")
        gdf.to_file(ruta_guardado, driver='GeoJSON')
        print(f"Capa guardada en: {ruta_guardado}")

    # 1. Carga de datos
    rutas = []
    contenido_drive = None

    # Opción para Google Colab
    try:
        from google.colab import files
        print("¿Desea subir un archivo desde su dispositivo? (s/n)")
        if input().lower() == 's':
            uploaded = files.upload()
            for fn, data in uploaded.items():
                rutas.append(fn)
                contenido_drive = data
    except ImportError:
        pass

    # Entrada de rutas tradicional
    if not rutas:
        input_rutas = input("Ingrese rutas/URLs separadas por comas: ").split(',')
        rutas = [r.strip() for r in input_rutas if r.strip()]

    # Procesar todas las rutas
    gdfs = []
    for ruta in rutas:
        gdf = cargar_shapefile(ruta, contenido_drive) if contenido_drive else cargar_shapefile(ruta)
        if gdf is not None:
            # Conversión CRS y formato
            if not gdf.crs:
                gdf.set_crs(epsg=4326, inplace=True)
            else:
                gdf = gdf.to_crs(epsg=4326)

            gdf = gdf.applymap(
                lambda x: x.isoformat() if isinstance(x, pd.Timestamp) else x
            )
            gdfs.append(gdf)

    # 2. Configuración de capas
    for i, gdf in enumerate(gdfs):
        print(f"\n--- Configuración de la capa {i+1} ---")

        # Selección de atributo para tooltip
        print("Atributos disponibles:", ", ".join(gdf.columns))
        while True:
            col_tooltip = input("Elija el atributo para mostrar en el mapa: ")
            if col_tooltip in gdf.columns:
                break
            print(f"¡Error! '{col_tooltip}' no existe. Intente nuevamente.")

        # Selección de estilo
        print("\nOpciones de estilo:")
        opcion_estilo = input(
            "1. Usar estilos del archivo\n"
            "2. Seleccionar estilo predefinido\n"
            "3. Crear estilo personalizado\n"
            "Opción: "
        )

        if opcion_estilo == '1':
            if {'color', 'fillColor'}.issubset(gdf.columns):
                style_func = lambda x: {
                    'color': x['properties']['color'],
                    'fillColor': x['properties']['fillColor']
                }
            else:
                style_func = lambda x: {'color': '#000000', 'fillColor': '#BF00BF'}

        elif opcion_estilo == '2':
            print(f"Estilos disponibles: {', '.join(ESTILOS_GUARDADOS.keys())}")
            estilo = input("Nombre del estilo: ").lower()
            style_func = lambda x: ESTILOS_GUARDADOS.get(estilo, ESTILOS_GUARDADOS['amenaza_media'])

        elif opcion_estilo == '3':
            color = input("Color del borde (hex): ") or '#000000'
            relleno = input("Color de relleno (hex): ") or '#FFFFFF'
            style_func = lambda x: {'color': color, 'fillColor': relleno}

        else:
            print("Opción inválida - Usando estilo por defecto")
            style_func = lambda x: {'color': '#000000', 'fillColor': '#BF00BF'}

        # Añadir capa al mapa
        nombre_capa = input(f"\nNombre para la capa {i+1}: ") or f"Capa {i+1}"
        # Guardar con ese nombre
        guardar_capa(gdf, nombre_capa)
        folium.GeoJson(
            gdf,
            name=nombre_capa,
            style_function=style_func,
            tooltip=folium.GeoJsonTooltip(fields=[col_tooltip])
        ).add_to(mapa)

    # Control de capas
    folium.LayerControl().add_to(mapa)
    print("\n¡Capas añadidas exitosamente!")

    return mapa

## FUNCION PARA AGREGAR TESELAS RASTER

In [ ]:
import folium

def agregar_mosaicos_control(mapa, seleccion=None):
    """
    Función para gestionar la selección y adición de mosaicos base a un mapa Folium.

    Parámetros:
    mapa (folium.Map): Mapa al que se agregarán los mosaicos
    seleccion (str/list/int, opcional): 'todos' para agregar todos, lista de índices o índice individual. Por defecto None.
    """

    # 1. Listado de mosaicos disponibles (built-in y custom)
    mosaicos = {
        "OpenStreetMap": {
            "tiles": "OpenStreetMap",
            "attr": "© OpenStreetMap contributors"
        },
        "Satélite (ESRI)": {
            "tiles": "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
            "attr": "Esri"
        },
        "Catastral (Breton)": {
            "tiles": "https://tile.openstreetmap.bzh/br/{z}/{x}/{y}.png",
            "attr": "OpenStreetMap contributors"
        },
        "Stamen Terreno": {
            "tiles": "Stamen Terrain",
            "attr": "Stamen"
        },
        "CartoDB Oscuro": {
            "tiles": "CartoDB dark_matter",
            "attr": "© CartoDB"
        },
        "CartoDB Positron": {
            "tiles": "CartoDB positron",
            "attr": "© CartoDB"
        },
        "Stamen Watercolor": {
            "tiles": "Stamen Watercolor",
            "attr": "Stamen"
        },
        "Stamen Toner": {
            "tiles": "Stamen Toner",
            "attr": "Stamen"
        },
        "Mapbox Bright": {
            "tiles": "Mapbox Bright",
            "attr": "© Mapbox"
        },
        "Mapbox Control Room": {
            "tiles": "Mapbox Control Room",
            "attr": "© Mapbox"
        }
    }

    # 2. Mostrar lista de mosaicos disponibles
    print("Mosaicos disponibles:")
    for i, nombre in enumerate(mosaicos.keys()):
        print(f"{i}: {nombre}")

    # 3. Determinar la selección según el parámetro
    indices = []
    if seleccion is not None:
        if seleccion == 'todos':
            indices = list(range(len(mosaicos)))
        elif isinstance(seleccion, int):
            indices = [seleccion]
        elif isinstance(seleccion, (list, tuple)):
            indices = seleccion
        else:
            print("Parámetro no válido. Se usará selección por entrada.")
            seleccion = None

    # Si no hay selección válida, solicitar entrada
    if not indices and seleccion is None:
        seleccion_input = input("\nIngresa los números de los mosaicos a agregar (separados por comas) o 'todos': ")
        indices = [int(idx.strip()) for idx in seleccion_input.split(",") if idx.strip().isdigit()]

    # 4. Validar y agregar mosaicos seleccionados
    nombres_mosaicos = list(mosaicos.keys())
    for idx in indices:
        if 0 <= idx < len(nombres_mosaicos):
            nombre = nombres_mosaicos[idx]
            config = mosaicos[nombre]

            folium.TileLayer(
                tiles=config["tiles"],
                attr=config["attr"],
                name=nombre,
                overlay=False,
                control=True,
                show=(idx == indices[0])  # Mostrar solo el primero
            ).add_to(mapa)

    # Agregar control de capas
    folium.LayerControl(position="topright", collapsed=False).add_to(mapa)

## FUNCION MARCO DE COORDENADAS UTM EN HTML

In [ ]:
def crear_mapa_con_marco_utm_dinamico(mapa_existente, lat_centro, lon_centro, zona_utm,
                                    ancho_marco=2000, intervalo=500,
                                    ancho_mapa=800, alto_mapa=600):
    """
    Marco dinamico de coordenadas UTM usalo;
    mapa_dinamico = crear_mapa_con_marco_utm_dinamico(
        mapa_existente=mapa,
        lat_centro=6.22204,
        lon_centro=-75.61399,
        zona_utm=18,
        ancho_marco=1500,
        intervalo=250
    )
    """
    # Configuración inicial de transformación
    crs_wgs84 = CRS("EPSG:4326")
    crs_utm = CRS(f"EPSG:326{zona_utm}")
    transformer = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True)
    centro_este, centro_norte = transformer.transform(lon_centro, lat_centro)

    # Generar HTML base manteniendo todos tus estilos
    marco_html = f"""
    <style>
        /* Mantener todos tus estilos CSS originales sin modificar */
        .map-container {{ position: relative; width: {ancho_mapa + 140}px; height: {alto_mapa + 140}px; padding: 20px; }}
        .utm-frame {{ position: relative; border: 2px solid #333; background: white; width: calc(100% - 80px); height: calc(100% - 80px); margin: 40px; }}
        .map-content {{ position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%); z-index: 1; width: {ancho_mapa}px; height: {alto_mapa}px; border: 1px solid #ddd; }}
        .coordenada {{ position: absolute; font-family: monospace; font-size: 14px; color: #333; background: rgba(255,255,255,0.9); padding: 4px; white-space: nowrap; z-index: 2; }}
        .linea-indicadora {{ position: absolute; background: #333; }}
        .coordenada-este.top {{ top: -34px; transform: translateX(-50%); }}
        .coordenada-este.top .linea-indicadora {{ width: 1px; height: 12px; top: 100%; left: 50%; }}
        .coordenada-este.bottom {{ bottom: -34px; transform: translateX(-50%); }}
        .coordenada-este.bottom .linea-indicadora {{ width: 1px; height: 12px; bottom: 100%; left: 50%; }}
        .coordenada-norte.left {{ left: -56px; transform: translateY(-50%); }}
        .coordenada-norte.left .texto-coordenada {{ display: inline-block; transform: rotate(-90deg) translateX(-8px); transform-origin: center; }}
        .coordenada-norte.left .linea-indicadora {{ width: 12px; height: 1px; left: 90%; top: 50%; }}
        .coordenada-norte.right {{ right: -52px; transform: translateY(-50%); }}
        .coordenada-norte.right .texto-coordenada {{ display: inline-block; transform: rotate(90deg); transform-origin: center; }}
        .coordenada-norte.right .linea-indicadora {{ width: 12px; height: 1px; right: calc(100% - 6px); top: 50%; }}
    </style>

    <div class="map-container">
        <div class="utm-frame" id="utm-frame">
            <!-- Las coordenadas se generarán dinámicamente aquí -->
        </div>

        <div class="map-content">
            {mapa_existente.get_root().render()}
        </div>
    </div>

    <!-- Incluir Proj4js para conversiones -->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/proj4js/2.8.0/proj4.js"></script>

    <script>
        // Configurar proyección UTM
        proj4.defs("EPSG:326{zona_utm}",
            "+proj=utm +zone={zona_utm} +datum=WGS84 +units=m +no_defs");

        // Elementos clave
        const mapa = {mapa_existente.get_name()};
        const marco = document.getElementById('utm-frame');

        // Función de actualización mejorada
        const actualizarCoordenadas = () => {{
            // Obtener límites actuales del mapa
            const bounds = mapa.getBounds();
            const center = mapa.getCenter();

            // Convertir límites a UTM
            const ne_utm = proj4('EPSG:4326', 'EPSG:326{zona_utm}', [bounds.getEast(), bounds.getNorth()]);
            const sw_utm = proj4('EPSG:4326', 'EPSG:326{zona_utm}', [bounds.getWest(), bounds.getSouth()]);

            // Calcular dimensiones del marco
            const ancho_total = ne_utm[0] - sw_utm[0];
            const alto_total = ne_utm[1] - sw_utm[1];

            // Generar HTML para coordenadas
            let htmlCoords = '';

            // Coordenadas Este (X)
            for(let x = sw_utm[0] + {intervalo}; x < ne_utm[0]; x += {intervalo}) {{
                const percentX = ((x - sw_utm[0]) / ancho_total) * 100;
                htmlCoords += `
                    <div class="coordenada coordenada-este top" style="left:${{percentX}}%">
                        ${{Math.round(x)}}
                        <div class="linea-indicadora"></div>
                    </div>
                    <div class="coordenada coordenada-este bottom" style="left:${{percentX}}%">
                        ${{Math.round(x)}}
                        <div class="linea-indicadora"></div>
                    </div>
                `;
            }}

            // Coordenadas Norte (Y)
            for(let y = sw_utm[1] + {intervalo}; y < ne_utm[1]; y += {intervalo}) {{
                const percentY = ((y - sw_utm[1]) / alto_total) * 100;
                htmlCoords += `
                    <div class="coordenada coordenada-norte left" style="top:${{percentY}}%">
                        <span class="texto-coordenada">${{Math.round(y)}}</span>
                        <div class="linea-indicadora"></div>
                    </div>
                    <div class="coordenada coordenada-norte right" style="top:${{percentY}}%">
                        <span class="texto-coordenada">${{Math.round(y)}}</span>
                        <div class="linea-indicadora"></div>
                    </div>
                `;
            }}

            marco.innerHTML = htmlCoords;
        }};

        // Manejadores de eventos
        mapa.on('moveend', actualizarCoordenadas);
        mapa.on('zoomend', actualizarCoordenadas);

        // Actualización inicial
        actualizarCoordenadas();

        // Asegurar actualización después de la carga
        setTimeout(actualizarCoordenadas, 500);
    </script>
    """

    # Crear mapa final manteniendo dimensiones
    mapa_final = folium.Map(width=ancho_mapa+140, height=alto_mapa+140)
    mapa_final.get_root().html.add_child(folium.Element(marco_html))

    return mapa_final